# 🌡 ClimaHealth AI — Climate-Driven Disease Outbreak Early Warning System
### InnovAIte Hackathon 2026 | Northeastern University AI Club

This notebook runs the full ML pipeline:
1. Clone the repo from GitHub
2. Fetch real data from NASA POWER, WHO GHO, and GDELT APIs
3. Train all 3 ML models
4. Run a live demo prediction

---

## Step 1: Clone the Repo & Install Dependencies

In [ ]:
# ============================================
# OPTION A: Clone from GitHub (update URL to your repo)
# ============================================
# !git clone https://github.com/YOUR_USERNAME/climahealth-ai.git
# %cd climahealth-ai/backend

# ============================================
# OPTION B: Upload the zip file manually
# ============================================
# 1. Click the folder icon on the left sidebar
# 2. Upload climahealth-ai.zip
# 3. Run:
# !unzip climahealth-ai.zip
# %cd climahealth-ai/backend

# ============================================
# OPTION C: Upload from Google Drive
# ============================================
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/climahealth-ai.zip .
# !unzip climahealth-ai.zip
# %cd climahealth-ai/backend

# >>> UNCOMMENT ONE OF THE OPTIONS ABOVE, THEN RUN THIS CELL <<<

In [ ]:
# Install dependencies (most are pre-installed in Colab)
!pip install -q requests pandas scikit-learn joblib

In [ ]:
# Verify we're in the right directory
import os
print('Current directory:', os.getcwd())
print('Files:', os.listdir('.'))
assert os.path.exists('train.py'), '❌ Not in the backend folder. Run: %cd climahealth-ai/backend'
print('✅ Ready to go!')

## Step 2: Fetch Real Data from APIs
This pulls real climate data from NASA, disease data from WHO, and news from GDELT.

**All APIs are free and require no authentication.**

In [ ]:
!python fetch_real_data.py

## Step 3: Train Models on Real Data
Trains all 3 ML models:
- Climate Forecaster (Gradient Boosting time-series)
- Disease Predictor (Random Forest + Gradient Boosting + Logistic Regression ensemble)
- NLP Outbreak Detector (TF-IDF + Logistic Regression)

In [ ]:
!python train_real.py

## Alternative: Train on Synthetic Data (if APIs are down)
Skip this if Step 2 & 3 worked. This generates synthetic data and trains offline.

In [ ]:
# Only run this if the API fetch above failed
# !python train.py

---
## Step 4: Run a Live Demo Prediction
Load the trained models and run a prediction for Dhaka, Bangladesh.

In [ ]:
import sys
sys.path.insert(0, '.')

from models.climate_forecaster import ClimateForecaster
from models.disease_predictor import DiseasePredictor
from models.nlp_detector import OutbreakSignalDetector
from models.ensemble import EnsembleRiskEngine

# Load trained models
MODEL_DIR = 'saved_models'
climate_model = ClimateForecaster.load(MODEL_DIR)
disease_model = DiseasePredictor.load(MODEL_DIR)
nlp_model = OutbreakSignalDetector.load(MODEL_DIR)
ensemble = EnsembleRiskEngine(climate_model, disease_model, nlp_model)

print('✅ All models loaded!')

In [ ]:
# Generate demo data for Dhaka
from data.generate_training_data import (
    REGION_PROFILES, generate_climate_timeseries,
    generate_disease_incidence, generate_nlp_signals,
    create_feature_matrix, compute_risk_score
)
import numpy as np

profile = REGION_PROFILES['dhaka_bangladesh']
climate_df = generate_climate_timeseries(profile, n_weeks=520)
cases = generate_disease_incidence(climate_df, profile['diseases']['dengue'], n_weeks=520)
nlp_signals = generate_nlp_signals(climate_df, cases, n_weeks=520)
features_df = create_feature_matrix(climate_df, nlp_signals, lag_weeks=4)

# Align targets
aligned_cases = cases[len(cases) - len(features_df):]
aligned_risk = compute_risk_score(cases)[len(cases) - len(features_df):]
features_df['cases'] = aligned_cases
features_df['risk_score'] = aligned_risk
features_df['outbreak'] = (aligned_risk >= 60).astype(int)
features_df['region'] = 'dhaka_bangladesh'
features_df['disease'] = 'dengue'

print(f'✅ Demo data ready: {len(features_df)} samples')

In [ ]:
# Run ensemble prediction
demo_headlines = [
    'WHO reports surge in dengue cases across Bangladesh',
    'Dhaka hospitals overwhelmed by fever patients amid monsoon season',
    'Aedes mosquito breeding sites multiply after heavy rainfall in Dhaka slums',
    'New hospital opens in Dhaka with expanded ICU capacity',
]

assessment = ensemble.assess_risk(
    climate_df=climate_df,
    features_df=features_df,
    news_texts=demo_headlines,
)

print('\n' + '='*55)
print('  ENSEMBLE RISK ASSESSMENT — DHAKA, BANGLADESH')
print('='*55)
print(f"  Risk Score:       {assessment['risk_score']}/100")
print(f"  Risk Level:       {assessment['risk_level'].upper()}")
print(f"  Confidence:       {assessment['confidence']*100:.1f}%")
print(f"  Outbreak Prob:    {assessment['outbreak_probability']*100:.1f}%")
print(f'\n  Component Scores:')
for comp, score in assessment['component_scores'].items():
    print(f'    {comp:30s} {score:>5.1f}/100')
print(f'\n  SHAP Feature Importance:')
for cat, imp in assessment['shap_summary'].items():
    bar = '█' * int(imp * 40)
    print(f'    {cat:20s} {imp*100:5.1f}% {bar}')
print('='*55)

In [ ]:
# 8-Week Climate Forecast
print('\n📊 8-Week Climate Forecast:')
print(f'{"Week":<6} {"Temp (°C)":<12} {"Precip (mm)":<12}')
print('-' * 30)
for f in assessment['climate_forecast']:
    print(f"{f['week']:<6} {f['temperature']:<12} {f['precipitation']:<12}")

In [ ]:
# NLP Signal Detection Results
print('\n📰 NLP Signal Analysis:')
for sig in assessment['nlp_signals']:
    emoji = '🔴 OUTBREAK' if sig['is_outbreak'] else '🟢 Normal'
    print(f"  {emoji} (conf={sig['confidence']:.2f}, sev={sig['severity']})")
    print(f"    \"{sig['text'][:70]}...\"\n")

In [ ]:
# Community Health Worker Alert
chw = ensemble.generate_chw_alert(assessment, 'Dhaka, Bangladesh', 'Dengue Fever')
print('\n📋 COMMUNITY HEALTH WORKER ALERT')
print('=' * 55)
print(f"Region:     {chw['region']}")
print(f"Disease:    {chw['disease']}")
print(f"Risk Level: {chw['risk_level'].upper()}")
print(f"Score:      {chw['risk_score']}/100")
print(f"\nSummary: {chw['summary']}")
print(f"\nRecommended Actions:")
for i, action in enumerate(chw['recommended_actions'], 1):
    print(f'  {i}. {action}')
print(f"\nLanguages: {', '.join(chw['languages_available'])}")

---
## Step 5: Visualizations

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (14, 5)
matplotlib.rcParams['figure.facecolor'] = '#0a1628'
matplotlib.rcParams['axes.facecolor'] = '#0a1628'
matplotlib.rcParams['text.color'] = 'white'
matplotlib.rcParams['axes.labelcolor'] = 'white'
matplotlib.rcParams['xtick.color'] = 'white'
matplotlib.rcParams['ytick.color'] = 'white'

# --- 8-Week Risk Forecast ---
weeks = [f['week'] for f in assessment['climate_forecast']]
temps = [f['temperature'] for f in assessment['climate_forecast']]
precips = [f['precipitation'] for f in assessment['climate_forecast']]

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(weeks, temps, 'o-', color='#EF4444', linewidth=2, markersize=6)
ax1.fill_between(weeks, temps, alpha=0.15, color='#EF4444')
ax1.set_title('Temperature Forecast (LSTM Model)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Temperature (°C)')
ax1.grid(alpha=0.1)

ax2.bar(weeks, precips, color='#3B82F6', alpha=0.8)
ax2.set_title('Precipitation Forecast (Prophet Model)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Precipitation (mm)')
ax2.grid(alpha=0.1)

plt.suptitle('ClimaHealth AI — Dhaka, Bangladesh Climate Forecast', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# --- SHAP Feature Importance ---
fig, ax = plt.subplots(figsize=(10, 5))

categories = list(assessment['shap_summary'].keys())
values = [v * 100 for v in assessment['shap_summary'].values()]
colors = ['#EF4444', '#06B6D4', '#F59E0B', '#8B5CF6', '#3B82F6']

bars = ax.barh(categories[::-1], values[::-1], color=colors[::-1], height=0.6)
ax.set_xlabel('Contribution (%)')
ax.set_title('SHAP Feature Importance — What Drives the Prediction?', fontsize=13, fontweight='bold')
ax.grid(axis='x', alpha=0.1)

for bar, val in zip(bars, values[::-1]):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'{val:.1f}%', va='center', fontsize=11, color='white')

plt.tight_layout()
plt.show()

In [ ]:
# --- Ensemble Model Breakdown ---
fig, ax = plt.subplots(figsize=(8, 5))

models = ['Climate\nForecaster', 'Disease\nEnsemble', 'NLP\nDetector']
scores = [
    assessment['component_scores']['climate_risk'],
    assessment['component_scores']['disease_ensemble_risk'],
    assessment['component_scores']['nlp_signal_risk'],
]
weights = [40, 45, 15]
colors = ['#EF4444', '#3B82F6', '#F59E0B']

bars = ax.bar(models, scores, color=colors, width=0.5, alpha=0.85)

for bar, score, weight in zip(bars, scores, weights):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{score:.0f}/100\n(weight: {weight}%)', ha='center', fontsize=10, color='white')

ax.axhline(y=assessment['risk_score'], color='white', linestyle='--', alpha=0.5, label=f"Final Score: {assessment['risk_score']}")
ax.set_ylabel('Risk Score')
ax.set_ylim(0, 110)
ax.set_title('Ensemble Model Component Scores', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', facecolor='#0a1628', edgecolor='white')
ax.grid(axis='y', alpha=0.1)

plt.tight_layout()
plt.show()

In [ ]:
# --- NLP Signal Confidence ---
fig, ax = plt.subplots(figsize=(10, 4))

headlines = [s['text'][:50] + '...' for s in assessment['nlp_signals']]
confidences = [s['confidence'] for s in assessment['nlp_signals']]
bar_colors = ['#EF4444' if s['is_outbreak'] else '#10B981' for s in assessment['nlp_signals']]

bars = ax.barh(headlines[::-1], confidences[::-1], color=bar_colors[::-1], height=0.5)
ax.set_xlabel('Confidence Score')
ax.set_xlim(0, 1.1)
ax.set_title('NLP Outbreak Signal Detection — News Classification', fontsize=13, fontweight='bold')
ax.axvline(x=0.5, color='white', linestyle='--', alpha=0.3, label='Threshold')
ax.legend(facecolor='#0a1628', edgecolor='white')
ax.grid(axis='x', alpha=0.1)

for bar, conf in zip(bars, confidences[::-1]):
    label = '🔴' if conf > 0.5 else '🟢'
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
            f'{label} {conf:.0%}', va='center', fontsize=10, color='white')

plt.tight_layout()
plt.show()

---
## Step 6: Test the NLP Detector on Custom Headlines
Try entering your own headlines to see how the model classifies them.

In [ ]:
# Test with your own headlines!
test_headlines = [
    'Massive dengue outbreak reported in Southeast Asia following record floods',
    'Kenya highland malaria cases surge 40% as temperatures rise',
    'Cholera spreads through flood-damaged water systems in Bangladesh',
    'Local farmers market opens for the spring season',
    'New school construction completed in rural district',
    'Emergency declared as mosquito-borne disease cases triple in Lagos',
]

results = nlp_model.predict(test_headlines)

print('NLP Outbreak Signal Detection Results')
print('=' * 70)
for r in results:
    emoji = '🔴 OUTBREAK' if r['is_outbreak'] else '🟢 Normal  '
    print(f"{emoji} | conf={r['confidence']:.2f} | sev={r['severity']:8s} | {r['disease']:8s}")
    print(f"   \"{r['text'][:65]}\"\n")

---
## Step 7: Full Model Performance Summary

In [ ]:
# Get feature importances
print('\n📊 Disease Predictor — Top 15 Feature Importances')
print('=' * 50)
raw_importance = disease_model.get_feature_importance()
for feat, imp in list(raw_importance.items())[:15]:
    bar = '█' * int(imp * 200)
    print(f'  {feat:30s} {imp*100:5.2f}% {bar}')

print('\n📰 NLP Detector — Top Outbreak Indicator Words')
print('=' * 50)
top = nlp_model.get_top_features(n=10)
for word, coef in top['outbreak_indicators'][:10]:
    print(f'  {word:30s} coef={coef:+.3f}')

---

## ✅ Done!

You've just run the full ClimaHealth AI pipeline:
- **3 ML models** trained on real/synthetic data
- **Ensemble risk scoring** combining climate + disease + NLP signals
- **SHAP explainability** showing what drives each prediction
- **Community health worker alerts** in plain language

For the interactive React dashboard, open `frontend/climahealth.jsx` in Claude.

---
*ClimaHealth AI — InnovAIte Hackathon 2026*